<a href="https://colab.research.google.com/github/ryyhan/fineTunedLLM/blob/main/GPT_2_Fine_Tuned(ConversationsData).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets evaluate torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
from datasets import load_dataset
from transformers import GPT2Tokenizer

In [4]:
#Loading the Dataset
import pandas as pd
import numpy as np

df = pd.read_csv("/content/conversation_data.csv")

In [5]:
df.head()

,Unnamed: 0,0,1,2
0,0,What kind of phone(s) do you guys have?,I have a pixel. It's pretty great. Much better...,Does it really charge all the way in 15 min?
1,1,I have a pixel. It's pretty great. Much better...,Does it really charge all the way in 15 min?,"Pretty fast. I've never timed it, but it's und..."
2,2,Does it really charge all the way in 15 min?,"Pretty fast. I've never timed it, but it's und...","cool. I've been thinking of getting one, my ph..."
3,3,What kind of phone(s) do you guys have?,Samsung Galaxy J1. It's my first cell phone an...,What do you think of it? Anything you don't like?
4,4,Samsung Galaxy J1. It's my first cell phone an...,What do you think of it? Anything you don't like?,I love it. I can't think of anything I don't l...


In [6]:
# getting just the conversations

convos = df['0'] +" "+ df['1']

In [7]:
# preprocessing and selecting a batch

np.sum(convos.isnull())
convos.dropna(inplace=True)
convos = convos.sample(n=1000)
convos.name = "conversations"

In [8]:
from datasets import Dataset

convos_dataset = Dataset.from_pandas(convos.to_frame())

print(convos_dataset)

Dataset({
    features: ['conversations', '__index_level_0__'],
    num_rows: 1000
})


In [9]:
from transformers import GPT2Tokenizer

# Load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Add a padding token to the tokenizer
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['conversations'], truncation=True, padding='max_length', max_length=512) # Changed 'text' to 'conversations'

tokenized_dataset = convos_dataset.map(tokenize_function, batched=True)

print(tokenized_dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset({
    features: ['conversations', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 1000
})


Setting up the tokenizer

In [10]:
# Set the labels for the model

def format_dataset(example):
    example['labels'] = example['input_ids'].copy()
    return example

tokenized_dataset = tokenized_dataset.map(format_dataset, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [11]:
print(type(tokenized_dataset))

<class 'datasets.arrow_dataset.Dataset'>


In [13]:
# Split the dataset into train and test sets
split_dataset = tokenized_dataset.train_test_split(test_size=0.2, seed=42)

# Access the train and test sets from the split_dataset
train_dataset = split_dataset['train']
test_dataset = split_dataset['test']

print(len(train_dataset))
print(len(test_dataset))

800
200


In [14]:
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments

# Load the pre-trained GPT-2 model
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [17]:
# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset,
)

In [18]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.120658
2,No log,0.105438
3,0.134600,0.099345


TrainOutput(global_step=600, training_loss=0.13166157722473146, metrics={'train_runtime': 500.4486, 'train_samples_per_second': 4.796, 'train_steps_per_second': 1.199, 'total_flos': 627100876800000.0, 'train_loss': 0.13166157722473146, 'epoch': 3.0})

In [29]:
# Save the best model

trainer.save_model('/content/drive/MyDrive/Colab Notebooks/SavedModels/GPT-2-Fine-Tuned(ConversationsData)/plainDataGpt2')

tokenizer.save_pretrained("/content/drive/MyDrive/Colab Notebooks/SavedModels/GPT-2-Fine-Tuned(ConversationsData)/tokenizer")

('/content/drive/MyDrive/Colab Notebooks/SavedModels/GPT-2-Fine-Tuned(ConversationsData)/tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/SavedModels/GPT-2-Fine-Tuned(ConversationsData)/tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/SavedModels/GPT-2-Fine-Tuned(ConversationsData)/tokenizer/vocab.json',
 '/content/drive/MyDrive/Colab Notebooks/SavedModels/GPT-2-Fine-Tuned(ConversationsData)/tokenizer/merges.txt',
 '/content/drive/MyDrive/Colab Notebooks/SavedModels/GPT-2-Fine-Tuned(ConversationsData)/tokenizer/added_tokens.json')

In [33]:
from transformers import pipeline

# Load the fine-tuned model
model = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/Colab Notebooks/SavedModels/GPT-2-Fine-Tuned(ConversationsData)/plainDataGpt2')
tokenizer = GPT2Tokenizer.from_pretrained('/content/drive/MyDrive/Colab Notebooks/SavedModels/GPT-2-Fine-Tuned(ConversationsData)/tokenizer')
text_generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

# Generate text for evaluation
generated_texts = []
for i in range(len(test_dataset)):
    # Extract the generated text from the dictionary and append it to the list
    generated_texts.append(text_generator(test_dataset[i]['conversations'], max_length=512)[0]['generated_text'])

# Calculate BLEU score
from evaluate import load
bleu = load('bleu')
results = bleu.compute(predictions=generated_texts, references=test_dataset['conversations'])
print('BLEU score:', results['bleu'])

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


BLEU score: 0.7728995659875085
